In [10]:
import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
MODEL_PATH = "model.pkl"

In [3]:
model = joblib.load(MODEL_PATH)

In [4]:
new_data = pd.read_csv('data/new.csv')
training_data = pd.read_csv('data/training.csv')

In [5]:
training_data.columns

Index(['Date', 'Ticker', 'Price', 'stock_p_change', 'SP500', 'SP500_p_change',
       'Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E',
       'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin',
       'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share',
       'Qtrly Revenue Growth', 'Gross Profit', 'EBITDA',
       'Net Income Avl to Common', 'Diluted EPS', 'Total Cash',
       'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity',
       'Current Ratio', 'Book Value Per Share', 'Operating Cash Flow', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding'],
      dtype='object')

In [6]:
columns_to_drop = ['% Held by Insiders1',
 '% Held by Institutions1', '5 Year Average Dividend Yield4', '52 Week High3',
 '52 Week Low3',
 '52 Week Range3', 'Avg Vol (10 day)3', 'Dividend Date3',
 'Ex-Dividend Date4',
 'Fiscal Year Ends',
 'Float8',
 'Forward Annual Dividend Rate4',
 'Forward Annual Dividend Yield4','Implied Shares Outstanding6',
 'Last Split Date3',
 'Last Split Factor2',
 'Levered Free Cash Flow  (ttm)',
 'Most Recent Quarter  (mrq)', 'Payout Ratio4', 'Quarterly Earnings Growth  (yoy)', 'S&P 500 52-Week Change3', 'Shares Short (6/28/2024)4',
 'Shares Short (8/31/2018)4',
 'Shares Short (prior month )4',
 'Shares Short (prior month 5/31/2024)4',
 'Shares Short (prior month 7/31/2018)4',
 'Shares Short4',
 'Short % of Float (6/28/2024)4',
 'Short % of Float (8/31/2018)4',
 'Short % of Float4',
 'Short % of Shares Outstanding (6/28/2024)4',
 'Short % of Shares Outstanding (8/31/2018)4',
 'Short % of Shares Outstanding4',
 'Short Ratio (6/28/2024)4',
 'Short Ratio (8/31/2018)4',
 'Short Ratio4', 'Trailing Annual Dividend Rate3',
 'Trailing Annual Dividend Yield3', 'Gross Profit  (ttm)']

new_data = new_data.drop(columns=columns_to_drop)

In [9]:
def convert(value):
    value = value.replace(',', '')
    if pd.isna(value) or value == '--':
        return np.nan
    elif '%' in value:
        return float(value.replace('%', '').strip())
    elif 'k' in value:
        return float(value.replace('k', '').strip()) * 1e3
    elif 'M' in value:
        return float(value.replace('M', '').strip()) * 1e6
    elif 'B' in value:
        return float(value.replace('B', '').strip()) * 1e9
    elif 'T' in value:
        return float(value.replace('T', '').strip()) * 1e12
    else:
        try:
            return float(value)
        except:
            print(value)


# Apply conversion functions to other columns
for col in new_data.columns:
    if col != 'Date' and new_data[col].dtype == object and col != 'Ticker':
        new_data[col] = new_data[col].apply(convert)

In [14]:
new_data = new_data.dropna()
X_train = training_data.drop(columns=['Date', 'Ticker', 'Price', 'stock_p_change', 'SP500', 'SP500_p_change', 'Gross Profit'])
X = new_data.iloc[:, 2:]
X.columns = X_train.columns
scaler = StandardScaler()
# Fit and transform the data
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [15]:
predictions = model.predict(X)

In [16]:
predictions

array([ True, False, False, False, False,  True, False,  True, False,
        True, False,  True, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
       False,  True, False,  True, False, False, False, False, False,
        True, False,  True, False,  True, False, False, False, False,
        True,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True, False, False, False,  True,  True, False,
       False, False,  True, False, False, False, False,  True, False,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
        True,  True,  True,  True,  True,  True, False, False,  True,
        True, False,  True, False,  True,  True, False, False,  True,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,

In [17]:
preds = pd.Series(predictions, name='Predictions')
#X_original = pd.DataFrame(scaler.inverse_transform(X), columns=X.columns)
merged = pd.concat([new_data.reset_index(drop=True), preds.reset_index(drop=True)], axis=1)


In [18]:
merged

,Date,Ticker,Market Cap,Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5yr expected),Price/Sales,Price/Book,Enterprise Value/Revenue,...,Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Beta (5Y Monthly),50-Day Moving Average3,200-Day Moving Average3,Avg Vol (3 month)3,Shares Outstanding5,Predictions
0,2024-07-23,MMM,5.809000e+10,6.927000e+10,16.99,14.24,1.90,1.78,11.94,2.12,...,451.17,1.64,8.79,6.170000e+09,1.00,101.38,88.22,4570000.0,5.533600e+08,True
1,2024-07-23,ABBV,3.065700e+11,3.625000e+11,51.67,15.80,0.46,5.66,38.29,6.66,...,924.06,0.94,4.53,2.269000e+10,0.62,166.25,161.89,5470000.0,1.770000e+09,False
2,2024-07-23,ACN,2.076500e+11,2.067100e+11,30.36,25.58,2.15,3.28,7.48,3.21,...,16.08,1.16,44.28,9.150000e+09,1.23,301.83,329.49,3690000.0,6.263800e+08,False
3,2024-07-23,ADBE,2.460100e+11,2.440300e+11,49.94,30.58,2.13,12.40,16.57,11.95,...,41.03,1.16,33.06,6.580000e+09,1.30,510.02,546.18,3360000.0,4.434000e+08,False
4,2024-07-23,AMD,2.519300e+11,2.489000e+11,225.90,45.66,1.43,11.16,4.48,10.92,...,5.34,2.64,34.73,1.700000e+09,1.68,163.78,152.81,51350000.0,1.620000e+09,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,2024-07-23,XEL,3.097000e+10,5.909000e+10,16.74,15.58,2.22,2.24,1.74,4.29,...,160.77,0.86,32.12,4.840000e+09,0.38,54.40,56.93,3450000.0,5.556400e+08,False
336,2024-07-23,XYL,3.378000e+10,3.521000e+10,48.38,32.89,2.36,4.10,3.31,4.43,...,23.27,1.67,42.04,9.450000e+08,1.04,139.08,120.45,1250000.0,2.424500e+08,False
337,2024-07-23,ZBRA,1.679000e+10,1.894000e+10,64.67,27.86,0.97,3.88,5.30,4.35,...,72.96,0.99,61.65,1.970000e+08,1.66,314.11,271.03,323090.0,5.142000e+07,False
338,2024-07-23,ZBH,2.270000e+10,2.826000e+10,23.93,13.68,1.66,3.09,1.80,3.79,...,47.63,1.66,61.24,1.500000e+09,1.01,111.89,117.37,1550000.0,2.057300e+08,False


: 

In [36]:
merged = merged.set_index('Date')
merged.to_csv('data/final.csv')

In [41]:
df = pd.read_csv('data/final.csv', index_col='Date')